# VMware EXSi Visualisations 2

In [1]:
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
from itertools import cycle
from IPython.core.display import HTML
from modules.Network import *
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

We get the json file with the information from: https://wmwaredata.s3.us-east-2.amazonaws.com/machines.json

In [3]:
url = 'https://wmwaredata.s3.us-east-2.amazonaws.com/machines.json'
r = requests.get(url, allow_redirects=True)
open('data/machines.json', 'wb').write(r.content)

10063

In [4]:
json_file = 'data/machines.json' 
df = pd.read_json(json_file)
df

,BootTime,CommitedSpace,ConnectionState,CreateDate,Folder,Guest,GuestId,HardwareVersion,IPAddress,Id,...,MemoryMB,Name,Notes,NumCpu,NumVirtualDisks,PowerState,UNCommitedSpace,Uid,VMHost,Version
0,2021-01-07 13:36:24.248327+00:00,23708312044,connected,2021-01-07 13:36:24.265589+00:00,[datastore1] OPNsense/OPNsense.vmx,OPNsense,freebsd12_64Guest,NaN,None,564d29f0-0330-cbe9-ff62-322e2a8e89b0,...,2048,OPNsense,dont_delete,2,1,poweredOn,23708312044,521d330d-04c0-11d5-be0e-7818cfbb2407,'vim.HostSystem:ha-host',vmx-14
1,None,17180140024,connected,2021-02-08 09:26:08.069277+00:00,[datastore1] centos-testing/centos-testing.vmx,centos-testing,centos8_64Guest,NaN,None,564de62b-3ba3-d95e-e043-7c3c3835c472,...,2048,centos-testing,dont_delete,1,1,poweredOff,17180140024,522fb36e-4a08-5a8f-7812-1b0d1ceb535b,'vim.HostSystem:ha-host',vmx-14
2,None,270840,connected,2021-02-08 08:48:16.045875+00:00,[datastore1] TestInstallation/TestInstallation...,TestInstallation,centos8_64Guest,NaN,None,564d403b-69bb-ef7e-07a2-e064461cea9f,...,2048,TestInstallation,dont_delete,1,1,poweredOff,270840,52de03ee-c829-7e4f-36e8-f5ec021d408a,'vim.HostSystem:ha-host',vmx-14
3,None,52428811647,connected,2021-01-21 17:50:38.270450+00:00,[datastore1] packer-proxy-rebuild/packer-proxy...,packer-proxy-rebuild,ubuntu64Guest,NaN,None,,...,8192,packer-proxy-rebuild,,2,1,poweredOff,52428811647,,'vim.HostSystem:ha-host',vmx-13
4,2021-02-04 13:36:34.225529+00:00,36593834190,connected,2021-02-04 13:33:31.307230+00:00,[datastore1] Controller/Controller.vmx,Controller,ubuntu64Guest,NaN,91.109.26.21,564d22d6-99de-8186-43ac-b01189fd0193,...,4096,Controller,,2,1,poweredOn,36593834190,527c8e8f-22a2-aa8e-f14c-943aae2a9769,'vim.HostSystem:ha-host',vmx-14
5,2021-02-04 12:54:16.039620+00:00,11834237420,connected,2021-02-04 13:43:03.387354+00:00,[datastore1] file-drop-1/file-drop-1.vmx,file-drop-1,ubuntu64Guest,NaN,10.42.0.1,564db201-59fd-7e3a-90be-bf81ddd5e8cd,...,2048,file-drop-1,,2,1,poweredOn,11834237420,523fa171-d3e6-090c-c37d-b9fdc28aaa7a,'vim.HostSystem:ha-host',vmx-13
6,2021-02-04 16:11:54.528815+00:00,15275868443,connected,2021-02-04 16:11:54.546228+00:00,[datastore1] ICAP2/ICAP2.vmx,ICAP2,ubuntu64Guest,NaN,91.109.26.19,564d1e80-e0fe-e6f5-dbfc-43d5ad229062,...,8192,ICAP2,,4,1,poweredOn,15275868443,52b45454-872d-cf3e-5b37-ec3792e4fc89,'vim.HostSystem:ha-host',vmx-07
7,None,6902784492,connected,2021-02-04 21:39:44.119473+00:00,[datastore1] IQT_ICAP_Test_020421/IQT_ICAP_Tes...,IQT_ICAP_Test_020421,ubuntu64Guest,NaN,None,564dd11c-291b-8450-abaa-d1fea268f34d,...,2048,IQT_ICAP_Test_020421,,1,1,poweredOff,6902784492,526bd8d8-e8e0-2f40-c226-d328007a14ac,'vim.HostSystem:ha-host',vmx-07
8,2021-02-07 10:38:40.660762+00:00,69698858335,connected,2021-02-07 10:38:40.682321+00:00,[datastore1] logtrixia/logtrixia.vmx,logtrixia,ubuntu64Guest,NaN,91.109.26.22,564d4d69-9038-90dd-030e-102a7251d021,...,16384,logtrixia,"Faisal, Monitoring Server, Demo",4,1,poweredOn,69698858335,,'vim.HostSystem:ha-host',vmx-13
9,2021-02-07 16:17:51.478830+00:00,18401468908,connected,2021-02-07 16:17:51.495868+00:00,[datastore1] test-srevice-ova/test-srevice-ova...,test-srevice-ova,ubuntu64Guest,NaN,172.18.0.1,564d22df-f8c8-6047-363c-ae9877bcde13,...,8192,test-srevice-ova,,2,1,poweredOn,18401468908,52510480-c357-6374-272e-50802f0bd98a,'vim.HostSystem:ha-host',vmx-07


## Option 1:  A subnetwork for each VM

In [5]:
# CREATE JSON FILE WITH NODES AND EDGES

network = Network()

network.groups = ['Name', 'GuestId', 'NumCpu', 'MemoryMB', 'PowerState', 'HardwareVersion', 'Version', 'ConnectionState']


#colors = []
#for k in range(len(df)):
#    colors.append(rgb2hex(plt.cm.plasma(13*k)))

pastels = []
for k in range(9):
    pastels.append(rgb2hex(plt.cm.Pastel1(k)))
for k in range(8):
    pastels.append(rgb2hex(plt.cm.Pastel2(k)))

colors = []
for i, color in zip(range(len(df)), cycle(pastels)):
    colors.append(color)
    
    
for i, color in zip(range(len(df)), colors):
    # NODES   
    labels = []
    
    label = df.iloc[i]['Name'] + "\n" + df.iloc[i]['Id']
    labels.append(label)
    network.add_node(label, color=color, shape='box', repeat_nodes=True)
    
    for group in network.groups[1:]:
        label = group + ':\n' + str(df.iloc[i][group])
        labels.append(label)
        network.add_node(label, color=color, repeat_nodes=True)
        
    #label = 'VMResourceConfiguration:\n' + wrap_by_word(df.iloc[i]['VMResourceConfiguration'], n=1)
    #labels.append(label)
    #network.add_node(label, color=color, shape='box', repeat_nodes=True)   
    
    # EDGES
    for label in labels[1:]:
        network.add_edge(network.nodesIds[labels[0]][-1], network.nodesIds[label][-1])
        

network.save_to_json('data/nodes_edges_3.json')

In [6]:
%%html
<div id="mynetwork"></div>

In [7]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});


require(['vis'], function(vis){ 
    
    var json = $.getJSON("data/nodes_edges_3.json")
      .done(function(data){
        var data = {
          nodes: data.nodes,
          edges: data.edges
        };
        var network = new vis.Network(container, data, options);
      });
       
    var options = {      
      width: '1200px',
      height: '1000px',
      locale: 'en',
      physics: true,
      interaction: {
        hover:true, 
        tooltipDelay: 300
      },        
      layout: {
        randomSeed: 1,
        improvedLayout: true,
      }
    };    
    
    var container = document.getElementById("mynetwork");
});

<IPython.core.display.Javascript object>

## Option 2: Connect all subnetworks to VMHost node

In [8]:
# CREATE JSON FILE WITH NODES AND EDGES

network = Network()

network.groups = ['Name', 'GuestId', 'NumCpu', 'MemoryMB', 'PowerState', 'HardwareVersion', 'Version', 'ConnectionState']

    
name_labels = []
for i, color in zip(range(len(df)), colors):
    # NODES   
    labels = []
    
    label = df.iloc[i]['Name'] + "\n" + df.iloc[i]['Id']
    labels.append(label)
    name_labels.append(label)
    network.add_node(label, color=color, shape='box', repeat_nodes=True)
    
    for group in network.groups[1:]:
        label = group + ':\n' + str(df.iloc[i][group])
        labels.append(label)
        network.add_node(label, color=color, repeat_nodes=True)
        
    #label = 'VMResourceConfiguration:\n' + wrap_by_word(df.iloc[i]['VMResourceConfiguration'], n=1)
    #labels.append(label)
    #network.add_node(label, color=color, shape='box', repeat_nodes=True)   
    
    # EDGES
    for label in labels[1:]:
        network.add_edge(network.nodesIds[labels[0]][-1], network.nodesIds[label][-1])
        
if is_unique(df['VMHost']):
    
    # Host Node
    host_label = 'VMHost:\n' + df.iloc[0]['VMHost']
    network.add_node(label=host_label, color="#5CD6D6", shape='box')
    
    # Conect Host node to all VMs
    for label in name_labels:
        network.add_edge(network.nodesIds[host_label][0], network.nodesIds[label][0], color="#5CD6D6", arrows="")       

network.save_to_json('data/nodes_edges_4.json')

In [9]:
%%html
<div id="mynetwork2"></div>

In [10]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});


require(['vis'], function(vis){ 
    
    var json = $.getJSON("data/nodes_edges_4.json")
      .done(function(data){
        var data = {
          nodes: data.nodes,
          edges: data.edges
        };
        var network = new vis.Network(container, data, options);
      });
       
    var options = {      
      width: '1200px',
      height: '1000px',
      locale: 'en',
      physics: true,
      interaction: {
        hover:true, 
        tooltipDelay: 300
      },        
      layout: {
        randomSeed: 1,
        improvedLayout: true,
      }
    };    
    
    var container = document.getElementById("mynetwork2");
});

<IPython.core.display.Javascript object>